In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


### 1.Create a list with the verses from the biblical Dead Sea Scrolls

#### 1.1 List of wanted motion verbs

In [2]:
motion_verbs = ['BW>[','HLK[','JY>[','JRD[','<BR[',
                '<LH[','CWB[','>TH[','BRX[','GJX[',
                'GLH[','GLL[','DXP[','DLG[','HWH[',
                'XWC[','XLP[','XSH[','VB<[','VWF[',
                'MHR[','MWC[','NGC[','NHR[','NWX[',
                'NWS[','NXT[','NVP[','NS<[','NPL[',
                'NTK[','SBB[','SWR[','SLQ[','<WZ[',
                '<WP[','PNH[','PF<[','YWP[','Y<D[',
                'QHL[','QPY[','QRB[','RWY[','FVH[',
                'CWX[','CWR=[','CVP[','CQQ[','T>R[',
                'T<H[']

#### 1.2 List of occurrences of the motion verbs

In [6]:
verbs = TFOb.all("scroll", DSS).filter(biblical=1).to_words.filter_in(lex=motion_verbs)
verbs

<word_5074 "J<WPP TLK JLK J<LHW B>J T<L T<L <LT JRD T<LH TGLH JY> >Y> TPNW GLH MBJ> TLKW [...] B>W JY>W JLK JLK JY> JY> JY> B>W J<LW LK TLKJ HLKTJ TLKJ >LK HLK >LK">

In [7]:
#set(verbs.book)

In [8]:
verb = verbs[100]
print(verb)
scroll = verb.to_scrolls.scroll[0]
verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
#print(verse)

Y>H


In [15]:
glyphes = " ".join(["".join(word.to_signs.glyphe) for word in verse.to_words])
glyphes = glyphes.replace("0", "").replace("'", "").upper()
glyphes

'W MJ JTN >T H <M H ZH B JDJ W >SJRH >T >BJMLK W J>MRW L >BJMLK RBH YB>K W Y>H '

In [10]:
print(verse.to_signs.glyphe)

['W', 'M', 'J', 'J', 'T', 'n', '>', 'T', 'H', '<', 'm', 'H', 'Z', 'H', 'B', 'J', 'D', "'", 'J', 'W', '>', 'S', 'J', 'R', 'H', '>', 'T', '>', 'B', 'J', 'M', 'L', 'k', 'W', 'J', '>', 'M', 'R', 'W', 'L', '>', 'B', 'J', 'M', 'L', 'k', 'R', 'B', 'H', 'Y', 'B', '>', "'", 'k', 'W', 'Y', '>', "'", 'H', '', '00']


In [17]:
def is_sign_unc(verse):
    """If a verse contains a missing or uncertain sign, returns True. Else, returns False."""
    
    unc_types = ['missing', 'unc']
    verse_sign_types = []
    
    for word in verse.to_words:

        word_signs = []

        for sign in word.to_signs:
            if sign.type[0] in unc_types:
                sign_type = "1"
            elif sign.type[0] == "cons":
                sign_type = "0"
            else:
                continue

            word_signs.append(sign_type)

        verse_sign_types.append("".join(word_signs))    
    return " ".join(verse_sign_types).rstrip().lstrip()

In [29]:
def check_signs_status(verse):
    
    verse_sign_types = []
    verse_consonantal = []
    
    for word in verse.to_words:
        
        word_signs_types = []
        word_cons = []
        
        for sign in word.to_signs:
            print(sign)
    
            if sign.unc[0]: # four possible values, 1, 2, 3 and None
                sign_type = "1"
                
            else:
                sign_type = "0"
                
            word_signs_types.append(sign_type)
            word_cons.append(sign.glyphe.upper())
            
        verse_sign_types.append("".join(word_signs_types))  
        verse_consonantal.append("".join(word_cons))
        
    return (" ".join(verse_sign_types).rstrip().lstrip(), " ".join(verse_consonantal).rstrip().lstrip())

In [30]:
print(verb, verse.book[0], verse.chapter[0], verse.verse[0], scroll)
print("Verse length: ", len(str(verse.to_signs)))
print("Sign_info length: ", len(is_sign_unc(verse)))
print(is_sign_unc(verse))
#print(check_signs_status(verse))
print(verse.to_signs)

 X4 f1 2 X4
Verse length:  0
Sign_info length:  5
1 0 1



In [31]:
def check_zero_amount(string):
    string = string.replace("1", "")
    return(string, len(string))

In [32]:
def count_zeros_length(s):
    # Replace all '1's with nothing
    s_without_ones = s.replace('1', '')

    # Replace multiple spaces with a single space
    s_cleaned = ' '.join(s_without_ones.split())

    # Return the length of the cleaned string
    return len(s_cleaned)

# Test the function with example strings
example_strings = ["000 001 11 000101 01101", "1 01 11 00 111000101 00 0000 00"]
lengths = [count_zeros_length(s) for s in example_strings]

# Print the lengths for each example string
for i, length in enumerate(lengths):
    print(f"Length of string {i+1}: {length}")

Length of string 1: 14
Length of string 2: 20


In [33]:
verse

<word_4 "">

In [34]:
#see 23: missing sign

correct_verses = 0
incorrect_verses = 0
zero_len_verses = 0
for verb in verbs:
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    glyphes = " ".join(["".join(word.to_signs.glyphe) for word in verse.to_words])
    glyphes = glyphes.replace("0", "").replace("'", "").replace("  ", " ").upper()
    
    if scroll and verse:
        signs = verse.to_signs
        
        if len(str(verse.to_signs)) != 0:

            if len(is_sign_unc(verse)) !=  len(str(verse.to_signs)):
                print("Verse length: ", len(str(verse.to_signs)))
                print("Glyphs length", len(glyphes))
                print("Zero count: ", count_zeros_length(is_sign_unc(verse)))
                print("Sign_info length: ", len(is_sign_unc(verse)))
                print(is_sign_unc(verse))
                print(verse.to_signs)
                print(glyphes)
                print("\n")
                incorrect_verses += 1

            elif len(is_sign_unc(verse)) ==  len(signs):
                correct_verses += 1
        else:
            zero_len_verses += 1

Verse length:  62
Glyphs length 64
Zero count:  62
Sign_info length:  65
0 00 000 0000 0 000 00000 00 0000 00 0000 0 00 000 00 000 0000  1
W >M >MR J>MR H <BD >HBTJ >T >DNJ >T >CTJ W >T BNJ L> >Y> XPCJ
W >M >MR J>MR H <BD >HBTJ >T >DNJ >T >#TJ W >T BNJ L> >Y> XP#J  


Verse length:  69
Glyphs length 79
Zero count:  69
Sign_info length:  74
0 00 1111 0000 0 0000 0 000 000 000 0000 0 00000 00 00 00 000 000 0 000 00
W L> TLKW B XQWT H GWJ >CR >NJ MCLX M PNJKM KJ >T KL >LH <FW W >QY BM
W L> # # # # TLKW  B XQWT H GWJ >#R >NJ M#LX M PNJKM KJ >T KL >LH <#W W >QY BM 


Verse length:  66
Glyphs length 68
Zero count:  66
Sign_info length:  68
0 000000 000 0 0000 0000 0000 0 000 0 00000 0000 000 00 00000 0000 1
W HQRBTM >CH L JHWH CB<T JMJM B JWM H CBJ<J MQR> QDC KL ML>KT <BDH
W HQRBTM >#H L JHWH #B<T JMJM B JWM H #BJ<J MQR> QD# KL ML>KT <BDH  


Verse length:  79
Glyphs length 82
Zero count:  79
Sign_info length:  81
1 0 00 000 0000 0 000 00000 0 000 00 000 00 000 0 0000 000 000000 00000 000 00

Verse length:  121
Glyphs length 122
Zero count:  121
Sign_info length:  122
000 0 000 000 0 000 0 0 000 00 0 0000 0 000 000 0000 000 00 0000 00 0 000 0 000000 0 000000  0 000 000 0 0000 0 000 0 0000
KJ> K >CR JRD H GCM W H CLG MN H CMJM W CMH LW> JCWB KJ> >M HRWH >T H >RY W HWLJDH W HYMJXH W NTN ZR< L ZWR< W LXM L >KWL
KJ> K >#R JRD H G#M W H #LG MN H #MJM W #MH LW> J#WB KJ> >M HRWH >T H >RY W HWLJDH W HYMJXH W NTN ZR< L ZWR< W LXM L >KWL 


Verse length:  121
Glyphs length 122
Zero count:  121
Sign_info length:  122
000 0 000 000 0 000 0 0 000 00 0 0000 0 000 000 0000 000 00 0000 00 0 000 0 000000 0 000000  0 000 000 0 0000 0 000 0 0000
KJ> K >CR JRD H GCM W H CLG MN H CMJM W CMH LW> JCWB KJ> >M HRWH >T H >RY W HWLJDH W HYMJXH W NTN ZR< L ZWR< W LXM L >KWL
KJ> K >#R JRD H G#M W H #LG MN H #MJM W #MH LW> J#WB KJ> >M HRWH >T H >RY W HWLJDH W HYMJXH W NTN ZR< L ZWR< W LXM L >KWL 


Verse length:  184
Glyphs length 191
Zero count:  184
Sign_info length:  188
0 00000 00 000 000 000000 0 0

Verse length:  94
Glyphs length 96
Zero count:  94
Sign_info length:  96
0 0000 000 000 000 00000 0000 000 000 0000 00000 0 000 0000 000 0000 0 0 000 000 0000 0 00 000 1
L >MWR LW> KJ> >RY MYRJM NBW> >CR LW> NR>H MLXMH W QWL CWPR LW> NCM< W L LXM LW> NR<B W CM NCB
L >MWR LW> KJ> >RY MYRJM NBW> >#R LW> NR>H MLXMH W QWL #WPR LW> N#M< W L LXM LW> NR<B W #M N#B  


Verse length:  76
Glyphs length 78
Zero count:  76
Sign_info length:  78
0 000 0 000 00 000 00000 000 0 000 0 000 0 000 0 000 0 000 0 000 0 000 0 000 1
W B>H W HKH >T >RY MYRJM >CR L MWT L MWT W >CR L CBJ L CBJ W >CR L XRB L XRB
W B>H W HKH >T >RY MYRJM >#R L MWT L MWT W >#R L #BJ L #BJ W >#R L XRB L XRB  


Verse length:  96
Glyphs length 98
Zero count:  96
Sign_info length:  98
0 000 0000 000 000 0 0000000 0 000 0 000 0 0000 0 000000 00000 000 000 00000 000 0000 0 00000000 1
M PNJ R<TM >CR <FW L HK<JSNJ L LKT L QVR L <BWD L >LWHJM >XRJM >CR LW> JD<WM HMH >TMH W >BWTJKMH
M PNJ R<TM >#R <#W L HK<JSNJ L LKT L QVR L <BWD L >LWHJM 

Verse length:  95
Glyphs length 101
Zero count:  95
Sign_info length:  97
0 00000 00 00 0000 01001 00 0 000 0 00 0 0000000 00 0 000 000 00 0000 0 000 0 000 0 000 0 0000000
W TW>MR LH BT PR<H HLK >T H JLD H ZH W HJNJQHW LJ W >NJ >TN >T FKRK W TQX H >CH H JLD W TJNJQHW
W TW>MR LH BT PR<H H # LK # >T H JLD H ZH W HJNJQHW LJ W >NJ >TN >T #KRK W TQX H >#H H JLD W TJNJQHW 


Verse length:  89
Glyphs length 93
Zero count:  89
Sign_info length:  94
0 0000 0 000 0 000000 0 00 0000 0 000 00 0 00 0 0000 000 0000 0 00000 00 00 0 000 0000000  1 1
W JGDL H JLD W TBJ>HW L BT PR<H W JHJ LH L BN W TQR> CMW MWCH W TW>MR KJ MN H MJM MCJTJHW
W JGDL H JLD W TBJ>HW L BT PR<H W JHJ LH L BN W TQR> #MW MW#H W TW>MR KJ MN H MJM M#JTJHW    


Verse length:  52
Glyphs length 54
Zero count:  52
Sign_info length:  55
0 000000 00 00000 00000 0 00000 0000 00000 000 0 000  1
W TBW>NH >L R<W>L >BJHN W JW>MR MDW< MHRTN BW> H JWM
W TBW>NH >L R<W>L >BJHN W JW>MR MDW< MHRTN BW> H JWM  


Verse length:  52
Glyphs length 54


Sign_info length:  398
0 0000 0000 000 000 0 000 000 0 000 0000 0 000 0 00  0 000 000 0 0000 00 0000 0 00000 0000 00 000 0000 000 00 000 0 000000 00 00 0000 0000 00 000 0000 0000 00 0 0 00000 0 0 000 0 0 0000 00 0 000 0 0000 000 000000 00 0 000 0 00 0 0000 000 00 0000 0 000000 0 000 0 000 00 000 000 000 000 000 0000 0 0000 0000 00 000 0000 000 00 000 0000 0 000 0 000 0 0000 0000 000 000 0 000 000 0 000 0000 0 000 0 00
W FMTJ PDWT BJN <MJ W BJN <MK L MXR JHJH H >WT H ZH W JB> MCH W >HRN >L PR<H W J>MRW >LJW KH >MR JHWH CLX >T <MJ W J<BDNJ KJ >M >JNK MCLX >T <MJ HNNJ MCLX BK W B <BDJK W B <MK W B BTJK >T H <RB W ML>W BTJ MYRJJM >T H <RB W GM H >DMH >CR HM <LJH W HPLJTJ B JWM H HW> >T >RY GCN >CR <MJ <MD <LJH L BLTJ HJWT CM <RB LM<N TD< KJ >NJ JHWH B QRB H >RY W FMTJ PDWT BJN <MJ W BJN <MK L MXR JHJH H >WT H ZH
W #MTJ PDWT BJN <MJ W BJN <MK L MXR JHJH H >WT H ZH W JB> M#H W >HRN >L PR<H W J>MRW >LJW KH >MR JHWH #LX >T <MJ W J<BDNJ KJ >M >JNK M#LX >T <MJ HNNJ M#LX BK W B <BDJK W B <MK W B 

...

Verse length:  51
Glyphs length 62
Zero count:  51
Sign_info length:  56
0000 0000 00000 0000 00001 00000 11101 000 0 00000 00000
HWRD C>WL G>WNK HMJT NBLT TXTJK < RMH W MKSJK TWL<H
HWRD #>WL G>WNK HMJT NBLT # TXTJK # # # < # RMH W MKSJK TWL<H 


Verse length:  54
Glyphs length 57
Zero count:  54
Sign_info length:  56
1 0 00 000 000 0 000 000 0 00000 000 000 0 00000 000 000
L KN GLH <MJ M BLJ D<T W KBWDW MTJ R<B W HMWNW YXH YM>
  L KN GLH <MJ M BLJ D<T W KBWDW MTJ R<B W HMWNW YXH YM> 


Verse length:  126
Glyphs length 128
Zero count:  126
Sign_info length:  129
00 00 000 00 0000 0 000 0 00 000 0000 0 0000 0 00000 0 0000 0 0000 00000 0 0000 0 000 00000 0 00 000 00 00 000 0 000 000 00000  1
<L KN XRH >P JHWH B <MW W JV JDW <LJW W JKHW W JRGZW H HRJM W THJH NBLTM K SWXH B QRB XWYWT B KL Z>T L> CB >PW W <WD JDW NVWJH
<L KN XRH >P JHWH B <MW W JV JDW <LJW W JKHW W JRGZW H HRJM W THJH NBLTM K SWXH B QRB XWYWT B KL Z>T L> #B >PW W <WD JDW NVWJH  


Verse length:  54
Glyphs length 56
Zero cou

Verse length:  75
Glyphs length 79
Zero count:  75
Sign_info length:  79
0 0 000 0 0000 0000 0 000000 0000 0 0 0000 0 0000 0 00 0 000 00000 0 000000 1 1
W B LKT H XJWT JLKW H >WPNJM >YLM W B HNF> H XJWT M <L H >RY JNF>W H >WPNJM
W B LKT H XJWT JLKW H >WPNJM >YLM W B HN#> H XJWT M <L H >RY JN#>W H >WPNJM    


Verse length:  75
Glyphs length 79
Zero count:  75
Sign_info length:  79
0 0 000 0 0000 0000 0 000000 0000 0 0 0000 0 0000 0 00 0 000 00000 0 000000 1 1
W B LKT H XJWT JLKW H >WPNJM >YLM W B HNF> H XJWT M <L H >RY JNF>W H >WPNJM
W B LKT H XJWT JLKW H >WPNJM >YLM W B HN#> H XJWT M <L H >RY JN#>W H >WPNJM    


Verse length:  67
Glyphs length 70
Zero count:  67
Sign_info length:  69
1 0 0000 0000 00000 0 000 00000 0000 0 0000 0 000 0 000 000 0000 0000
L QYBJ HRJM JRDTJ H >RY BRXJH B<DJ L <WLM W T<L M CXT XJJ JHWH >LHJ
  L QYBJ HRJM JRDTJ H >RY BRXJH B<DJ L <WLM W T<L M #XT XJJ JHWH >LHJ 


Verse length:  67
Glyphs length 70
Zero count:  67
Sign_info length:  69
1 0 0000 0000 00000 0

Verse length:  43
Glyphs length 48
Zero count:  43
Sign_info length:  45
00000 000 110 000 0000 000000 000 0000 0 0000
>PPNJ MJM D NPC THWM JSBBNJ SWP XBWC L R>CJ
>PPNJ MJM # # D NP# THWM JSBBNJ SWP XBW# L R>#J 


Verse length:  37
Glyphs length 44
Zero count:  37
Sign_info length:  43
0 0000 0000 0 000 00 00000 0 000 0000 1 1 1
W JQWM JWNH W JLK >L NJNWH K DBR JHWH
W JQWM JWNH W JLK >L NJNWH K DBR JHWH   #   


Verse length:  101
Glyphs length 104
Zero count:  101
Sign_info length:  103
1 0 00 000000 0000 0 00 000000 00000 0 00 0 00 00000 0000 0000 00 0 0000 0000 0000 0 00 0000 0000 00000
W KL PSJLJH JKTW W KL >TNNJH JFRPW B >C W KL <YBJH >FJM CMMH KJ M >TNN ZWNH QBYH W <D >TNN ZWNH JCWBW
  W KL PSJLJH JKTW W KL >TNNJH J#RPW B ># W KL <YBJH >#JM #MMH KJ M >TNN ZWNH QBYH W <D >TNN ZWNH J#WBW 


Verse length:  105
Glyphs length 108
Zero count:  105
Sign_info length:  107
1 0 00 00 000 0000 0000 000 00 0 00000 0 000 000 000 000 00000 0 00 000000000 0 000 0000 0000 00 00 000 000
L KN KH >

Verse length:  93
Glyphs length 95
Zero count:  93
Sign_info length:  95
0 0000 0000 00 0 000 0 00000 00000 00 0000 0000 000000 00 000 00000 000 000 0 000 00000 0 000 1
W JCQD JHWH <L H R<H W JBJ>H <LJNW KJ YDJQ JHWH >LHJNW <L KWL M<FJW >CR <FH W LW> CM<NW B QLW
W J#QD JHWH <L H R<H W JBJ>H <LJNW KJ YDJQ JHWH >LHJNW <L KWL M<#JW >#R <#H W LW> #M<NW B QLW  


Verse length:  105
Glyphs length 108
Zero count:  105
Sign_info length:  107
1 000 000 0 000 00 00 000 0000 0000 000000 0 00 0000 00000 00 000 000 0000 0000 000000 0000 0 00000 0 00000
QRB >TH W CM< >T KL >CR J>MR JHWH >LHJNW W >T TDBR >LJNW >T KWL >CR JDBR JHWH >LHJNW >LJK W CM<NW W <FJNW
  QRB >TH W #M< >T KL >#R J>MR JHWH >LHJNW W >T TDBR >LJNW >T KWL >#R JDBR JHWH >LHJNW >LJK W #M<NW W <#JNW 


Verse length:  64
Glyphs length 67
Zero count:  64
Sign_info length:  66
0 000 000 0000 00 00000 00 0 000 00 0 000 0000 0 000 00 00000 00 1
B BJT >XD J>KL L> TWYJ> MN H BJT MN H BFR XWYH W <YM L> TCBRW BW
B BJT >XD J>KL L> TWYJ> MN H BJT

Verse length:  51
Glyphs length 58
Zero count:  51
Sign_info length:  57
1 1 1 0 00000 0000 0000 00000 0000 0 00000 0000 0 0000000
W YPWNK TML> BVNM JFB<W BNJM W HNJXW JTRM L <WLLJHM
  #   W YPWNK TML> BVNM J#B<W BNJM W HNJXW JTRM L <WLLJHM 


Verse length:  34
Glyphs length 36
Zero count:  34
Sign_info length:  37
0 000 0000 0000 0000 000 0 0000 00  1
M NGH NGDW <BJW <BRW BRD W GXLJ >C
M NGH NGDW <BJW <BRW BRD W GXLJ >#  


Verse length:  123
Glyphs length 136
Zero count:  123
Sign_info length:  133
0 000 000 0000 00000 000000 1 100001 1 0 00000 0 0000 0 00  00 0 00000 000 00 00000 000001 1 0 000 0000 0000 00000 0 0000 0 0000 0 00
B CB< NPC JRDW >BTJK MYRJMH >MWR K KWKBJ H CMJM L RB KJ B CB<JM NPC RD >BTJK MYRJM W <TH FMJT JHWH >LHJK K KKBJ H CMJM L RB
B #B< NP# JRDW >BTJK MYRJMH   # >MWR #   K KWKBJ H #MJM L RB KJ B #B<JM NP# RD >BTJK MYRJM #   W <TH #MJT JHWH >LHJK K KKBJ H #MJM L RB 


Verse length:  46
Glyphs length 48
Zero count:  46
Sign_info length:  48
0 000 000 0000 1 00 0000

In [35]:
print(f"Correct verses : {correct_verses}.\nIncorrect verses: {incorrect_verses}.\nZero sign verses: {zero_len_verses}")

Correct verses : 11.
Incorrect verses: 497.
Zero sign verses: 86
